# Testing
## Assertions

In [1]:
assert True == True

In [2]:
assert True == False

AssertionError: 

In [3]:
def mean(num_list):
    assert len(num_list) != 0
    return sum(num_list) / len(num_list)

In [4]:
mean([1, 2, 3, 4, 5])

3.0

In [5]:
mean([])

AssertionError: 

## NumPy
### assert_almost_equal

In [6]:
import numpy as np
from numpy.testing import assert_almost_equal

In [7]:
x = 1.9999998
assert_almost_equal(x, 2, decimal=4)

In [8]:
y = 1.9995
assert_almost_equal(y, 2, decimal=4)

AssertionError: 
Arrays are not almost equal to 4 decimals
 ACTUAL: 1.9995
 DESIRED: 2

### assert_all_close

In [9]:
from numpy.testing import assert_allclose

a = np.array([2.0, 1.9999, 2.0001 ])
assert_allclose(a, 2, rtol=0.003, atol=0)


In [10]:
b = np.array([2.0, 1.9999, 2.01 ])
assert_allclose(b, 2, rtol=0.003, atol=0)


AssertionError: 
Not equal to tolerance rtol=0.003, atol=0

(mismatch 33.33333333333333%)
 x: array([ 2.    ,  1.9999,  2.01  ])
 y: array(2)

## Exceptions

In [11]:
def mean(num_list):
    if len(num_list) == 0:
        raise Exception('The algebraic mean of an empty list is undefined.')
    else:
        return sum(num_list) / len(num_list)


In [12]:
mean([])

Exception: The algebraic mean of an empty list is undefined.

Alternatively:

In [13]:
def mean(num_list):
    try:
        return sum(num_list) / len(num_list)
    except ZeroDivisionError as detail:
        msg = 'The algebraic mean of an empty list is undefined.'
        raise ZeroDivisionError(str(detail) + '\n' + msg)

In [14]:
mean([])

ZeroDivisionError: division by zero
The algebraic mean of an empty list is undefined.

## UnitTests

In [15]:
from mean import *

def test_ints():
    num_list = [1, 2, 3, 4, 5]
    observed = mean(num_list)
    expected = 3
    assert observed == expected

In [16]:
test_ints()

Open files `mean.py` and `test_mean.py` from the testing-files directory in separate tabs.
Also open an **additional** Terminal (or Git Bash) window. (**Don't stop the Jypyter Notebook server**)

Import the test_mean package and run each test manually.

**That** was tedious!

## Running Tests with pytest /  nose

### Everyone who's using their own computer
Run the following command on the terminal run **`conda install pytest`**  and **`py.test`**:

```shell
$ conda install pytest
Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment C:\Users\ostueker\SWC-conda:

The following NEW packages will be INSTALLED:

    py:     1.4.33-py36_0
    pytest: 3.0.7-py36_0

Proceed ([y]/n)? y

py-1.4.33-py36 100% |###############################| Time: 0:00:00 529.05 kB/s
pytest-3.0.7-p 100% |###############################| Time: 0:00:00 852.45 kB/s

$ py.test
============================= test session starts =============================
platform win32 -- Python 3.6.1, pytest-3.1.0, py-1.4.33, pluggy-0.4.0
rootdir: C:\Users\ostueker\Desktop\CMSC6950\testing-files, inifile:
collected 5 items

test_mean.py ....F

================================== FAILURES ===================================
________________________________ test_complex _________________________________

    def test_complex():
        # given that complex numbers are an unordered field
        # the arithmetic mean of complex numbers is meaningless
        num_list = [2 + 3j, 3 + 4j, -32 - 2j]
        obs = mean(num_list)
        exp = NotImplemented
>       assert obs == exp
E       assert (-9+1.6666666666666667j) == NotImplemented

test_mean.py:34: AssertionError
===================== 1 failed, 4 passed in 9.65 seconds ======================
```

### Everyone who's using a Mac Mini:
Run the following command on the terminal:
```shell
$ nosetests
....F
======================================================================
FAIL: test_mean.test_complex
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\ostueker\SWC-conda\lib\site-packages\nose\case.py", line 197, in runTest
    self.test(*self.arg)
  File "C:\Users\ostueker\Desktop\CMSC6950\testing-files\test_mean.py", line 34, in test_complex
    assert obs == exp
AssertionError

----------------------------------------------------------------------
Ran 5 tests in 9.480s

FAILED (failures=1)
```

### Fix the failing code
```python
def mean(num_list):
    try:
        result = sum(num_list)/len(num_list)
        if isinstance( result, complex):
            return NotImplemented
        else:
            return result
    except ZeroDivisionError as detail :
        msg = "The algebraic mean of an empty list is undefined."
        msg += "Please provide a list of numbers."
        raise ZeroDivisionError(detail.__str__() + "\n" +  msg)
    except TypeError as detail :
        msg = "The algebraic mean of an non-numerical list is undefined."
        msg += "Please provide a list of numbers."
        raise TypeError(detail.__str__() + "\n" +  msg)
```

**Now run py.test or nosetests again**

## Edge Cases and Corner Cases
### Edge Cases
Consider the Fibbonacci Sequence

In [17]:
def fib(n):
    if n==0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

`test_fib.py`:

```python
from mod import fib

def test_fib0():
    # test edge 0
    obs = fib(0)
    assert obs == 1

def test_fib1():
    # test edge 1
    obs = fib(1)
    assert obs == 1

def test_fib6():
    # test internal point
    obs = fib(6)
    assert obs == 13
```

### Corner Cases

Consider:
```python
import numpy as np

def sinc2d(x, y):
    if x == 0.0 and y == 0.0:
        return 1.0
    elif x == 0.0:
        return np.sin(y) / y
    elif y == 0.0:
        return np.sin(x) / x
    else:
        return (np.sin(x) / x) * (np.sin(y) / y)
```

Then a set of tests could look like:

```python
import numpy as np

from mod import sinc2d

def test_internal():
    exp = (2.0 / np.pi) * (-2.0 / (3.0 * np.pi))
    obs = sinc2d(np.pi / 2.0, 3.0 * np.pi / 2.0)
    assert obs == exp

def test_edge_x():
    exp = (-2.0 / (3.0 * np.pi))
    obs = sinc2d(0.0, 3.0 * np.pi / 2.0)
    assert obs == exp

def test_edge_y():
    exp = (2.0 / np.pi)
    obs = sinc2d(np.pi / 2.0, 0.0)
    assert obs == exp
```

But what if both x and y are 0.0?


## Integration and Regression Tests

<http://katyhuff.github.io/python-testing/07-integration.html>

### Integration Tests
While **Unit Tests** are meant to test the smallest units of a program in isolation, 
**Integration Tests** will tests several units in concert and programs as a whole.

### Regression Tests

Sometimes we don't know the *correct* result for a calculation.
We therefore assume that the past is "correct". 

We run the tests often (e.g. after every code change) and expect to get the same results as ealier.
We will be notified if the program returns results that are different from those we used to get.

After discovering a bug, one should also write a test to detect if it re-appears at a later time.

## Continous Integration
<http://katyhuff.github.io/python-testing/08-ci.html>

Services like Travis-CI can be set up to run the whole test suite after code changes have been pushed to a Version Control Repository (e.g. at GitHub).  They will notify the author (and maintainer) via email if tests fail.

## Test Driven Development
<http://katyhuff.github.io/python-testing/09-tdd.html>

Test First! Test First!

![Cycle of Test Driven Development](https://raw.githubusercontent.com/mjhea0/flaskr-tdd/master/tdd.png)

## Fixtures
<http://katyhuff.github.io/python-testing/10-fixtures.html>

Functions for setup and teardown.

e.g.: 
* Start database service and before the tests and shut it down afterwards.
* Create specific data structures before the test.
* Create files with certain content and clean up afterwards.